<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Model Serving with Iguazio

In [2]:
import mlrun

project = mlrun.get_or_create_project("default", "./")

> 2023-03-03 09:05:18,639 [info] loaded project default from MLRun DB


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

### Define our model server

In [3]:
# mlrun: start-code

from cloudpickle import load
import numpy as np
from typing import List
import mlrun

class ClassifierModel(mlrun.serving.V2ModelServer):

    def load(self):
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

# mlrun: end-code

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

### Create our serving function

In [4]:
serving_fn = mlrun.code_to_function(name='ClassifierService',
                                    filename="model_serving.ipynb", 
                                    kind='serving',
                                    image='mlrun/mlrun')

serving_fn.spec.default_class = 'ClassifierModel'

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

### Add our model to the serving function

In [5]:
model_file = project.get_artifact_uri('my_model') 

serving_fn.add_model('my_model', model_path=model_file)

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

### Test locally with a mock server

In [6]:
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''

server = serving_fn.to_mock_server()

server.test("/v2/models/my_model/infer", body=my_data)

> 2023-03-03 09:05:41,114 [info] model my_model was loaded
> 2023-03-03 09:05:41,116 [info] Loaded ['my_model']


X does not have valid feature names, but RandomForestClassifier was fitted with feature names


{'id': '2647d5eafc034ab7b3647235251cbb69',
 'model_name': 'my_model',
 'outputs': [0, 2]}

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

### Deploy the model to the cluster

In [7]:
function_address = serving_fn.deploy()

> 2023-03-03 09:05:51,965 [info] Starting remote function deploy
2023-03-03 14:05:52  (info) Deploying function
2023-03-03 14:05:52  (info) Building
2023-03-03 14:05:52  (info) Staging files and preparing base images
2023-03-03 14:05:52  (info) Building processor image
2023-03-03 14:06:47  (info) Build complete
2023-03-03 14:07:43  (info) Function deploy complete
> 2023-03-03 09:07:43,552 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-default-classifierservice.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-classifierservice-default.default-tenant.app.us-sales-350.iguazio-cd1.com/']}


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

### Using the live model server

In [8]:
print(function_address)
!curl $function_address

http://default-classifierservice-default.default-tenant.app.us-sales-350.iguazio-cd1.com/
{"name": "ModelRouter", "version": "v2", "extensions": []}

In [9]:
serving_fn.invoke('/v2/models/my_model/infer', my_data)

> 2023-03-03 09:08:08,476 [info] invoking function: {'method': 'POST', 'path': 'http://default-classifierservice-default.default-tenant.app.us-sales-350.iguazio-cd1.com//v2/models/my_model/infer'}


{'id': 'f311c1c7-6c94-45e8-81c1-16bd8534adaf',
 'model_name': 'my_model',
 'outputs': [0, 2]}